# Simulation Analyses
This notebook contains the code to reproduce analyses related to the computational simulations for the main paper version of Visual Scoping. 

> What should we expect from following different strategies of taking future costs into account?

This notebook is organized according to the topic sentences, which can be found in the tex file of the manuscript.
For human data and model fitting, see other notebooks in this folder.

Requires:

* `.pkl` files generated by `experiment/simulate_agents.*.py` (which in turn need `.pkl` files that hold the cached sequences)

## Setup

In [ ]:
# set up imports
import os
import sys
__file__ = os.getcwd()
proj_dir =  os.path.dirname(os.path.realpath(__file__))
sys.path.append(proj_dir)
utils_dir = os.path.join(proj_dir,'utils')
sys.path.append(utils_dir)
analysis_dir = os.path.join(proj_dir,'analysis')
analysis_utils_dir = os.path.join(analysis_dir,'utils')
sys.path.append(analysis_utils_dir)
agent_dir = os.path.join(proj_dir,'model')
sys.path.append(agent_dir)
agent_util_dir = os.path.join(agent_dir,'utils')
sys.path.append(agent_util_dir)
experiments_dir = os.path.join(proj_dir,'experiments')
stim_dir = os.path.join(proj_dir,'stimuli')
sys.path.append(stim_dir)
sys.path.append(experiments_dir)
df_dir = os.path.join(proj_dir,'results/dataframes')

In [ ]:
from model.Subgoal_Planning_Agent import *
import utils.blockworld as bw
import utils.blockworld_library as bl
from stimuli.tower_generator import TowerGenerator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

from scipy import stats
from scipy.stats import sem as sem
import math

import itertools

import random
from tqdm import tqdm
import p_tqdm

from IPython.display import clear_output

In [ ]:
import re
import ast
def str2array(s):
    #strip "array" and parentheses
    s=re.sub('\[array\(', '', s.strip())
    s=re.sub('\)]', '', s.strip())
    # Remove space after [
    s=re.sub('\[ +', '[', s.strip())
    # Replace commas and spaces
    s=re.sub('[,\s]+', ', ', s)
    return np.array(ast.literal_eval(s))

def str2list(s):
    if s is np.nan: return s
    #strip "array" and parentheses
    s=re.sub('\[array\(', '', s.strip())
    s=re.sub('\)]', '', s.strip())
    # Remove space after [
    s=re.sub('\[ +', '[', s.strip())
    # Replace commas and spaces
    s=re.sub('[,\s]+', ', ', s)
    return list(ast.literal_eval(s))

In [ ]:
#helper function for pd.agg
def item(x):
    return x.tail(1).item()

In [ ]:
#inline plots
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Plot styling:

In [ ]:
plt.rcParams["figure.figsize"] = (7,7)
plt.rcParams.update({'font.size': 26})

In [ ]:
from matplotlib import rc
# plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica']
rc('text.latex', preamble=r'\usepackage{tgheros} \usepackage{newtxsf} \renewcommand{\familydefault}{\sfdefault} \usepackage{mathastext}') #sets the font via latex preamble—only way to autoset tick labels?

In [ ]:
#display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 20)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.min_rows', 12)

## Loading data
Let's load the results of the experiment. Provide the path to a folder containing the dataframes genereated by `simulate_agents.*.py`

In [ ]:
folder_path = "/Users/felixbinder/Cloud/Grad School/Fan Lab/Block Construction/tools_block_construction/results/dataframes/simulated_subgoal_agents__4"

In [ ]:
csv_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
print("Got {} csv files".format(len(csv_paths)))

In [ ]:
pkl_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.pkl')]
print("Got {} pkl files".format(len(pkl_paths)))

In [ ]:
# #load all experiments as one dataframe from CSV
# dfs = [pd.read_csv(os.path.join(df_dir,l)) for l in csv_paths]
# print("Read {} dataframes:".format(len(dfs)))
# # merge dfs
# df = pd.concat(dfs)
# print("Merged dataframes: {}".format(df.shape))

In [ ]:
# FOR DEVELOPMENT
# cut short the file list
# pkl_paths = pkl_paths[:1]

In [ ]:
# load all experiments as one dataframe from pickles—this can take a long time and use a lot of memory
dfs = [pd.read_pickle(os.path.join(df_dir,l)) for l in tqdm(pkl_paths)]
print("Read {} dataframes".format(len(dfs)))
# merge dfs
df = pd.concat(dfs)
print("Merged dataframes: {}".format(df.shape))

In [ ]:
# do a few things to add helpful columns and such
# use either solution_cost or states_evaluated as cost
df['cost'] = np.maximum(df['partial_solution_cost'].fillna(0),
                        df['states_evaluated'].fillna(0))
# do the same for total cost
df['total_cost'] = np.maximum(df['all_sequences_planning_cost'].fillna(
    0), df['states_evaluated'].fillna(0))

In [ ]:
# add world size—note that world size is in pixels, not number of blocks in the reference solution
df['world_size'] = df['_world'].apply(lambda x: np.sum(x.full_silhouette > 0))

In [ ]:
# we also want the number of subgoals considered
df['all_sequences_count'] = df['_all_subgoal_sequences'].apply(lambda x: len(x) if type(x) is not float else np.nan)

In [ ]:
#fdf holds final rows for every run
fdf = df.groupby('run_ID').agg({
        'agent': 'first',
        'agent_type': item,
        'c_weight': 'first',
        'label': 'first',
        'world': item,
        'action': 'count',
        'blockmap': 'last',
        'states_evaluated': ['sum', 'mean', sem],
        'planning_cost': ['sum', 'mean', sem], 
        'partial_planning_cost': ['sum', 'mean', sem], # the planning cost of the sequence as far as acted
        'partial_solution_cost': ['sum', 'mean', sem],
        'solution_cost': ['sum', 'mean', sem],
        'all_sequences_planning_cost': ['sum', 'mean', sem],
        'all_sequences_count': 'sum',
        # 'num_subgoals_acted': ['sum', 'mean', sem],
        'perfect': 'last',
        'planning_step': 'max',
        'cost': ['sum', 'mean', sem],
        'total_cost': ['sum', 'mean', sem],
        'world_size': item,
})

#flatten the dataframe to remove multi-index for next groupby
fdf.columns = [' '.join(col).strip() for col in fdf.columns.values]
fdf.reset_index(inplace=True)
# What is the number of blocks used?
# fdf['num_blocks'] = fdf['blockmap last'].apply(lambda x: np.max(str2array(x)))
#store note order as categorical to ensure sort
# fdf['note item'] = pd.Categorical(fdf['note item'],NOTE_ORDER) #restore the order of column

## Helper functions

In [ ]:
# extraction functions
def CI95(data): #this is NOT bootstrapped
#     return st.t.interval(alpha=0.95,df=len(data)-1,loc=np.mean(data),scale=st.sem(data))
    return tuple(np.percentile(data,[2.5,97.5]))

def names(list_names):
    if list_names is np.nan: return np.nan
    return [g for g in list_names if g is not np.nan]

### Bootstrapping

In [ ]:
def bootstrap_over_runs(df, column, stat_function = np.mean, CIs = [2.5,97.5], iterations = 1000, show_tqdm = True):
    """Bootstrap by choosing individual runs across towers (`run_ID`). 
    The given df should only contain rows for the relevant algorithm/conditions.
    Returns mean and CI of mean."""
    measurements = np.zeros(iterations)
    for i in tqdm(range(iterations), leave=False, disable = not show_tqdm):
        #sample with replacement
        run = df.sample(frac=1, replace=True)[column]
        #save that run
        measurements[i] = stat_function(run)
    #compute mean and CI over measurements
    return np.mean(measurements),np.percentile(measurements, CIs), stat_function(df[column])

### Plotting

In [ ]:
def get_shape(label):
    if label == "No Subgoal": return "o"
    if label == "Myopic": return "s"
    if label == "Lookahead": return "^"
    if label == "Lookahead 2": return "v"
    if "Full Decomp" in label: return "d"
    return "o"

In [ ]:
def get_colors(label):
    if label == "Myopic":
        return 'limegreen'
    if label == "Lookahead 1":
        return 'cornflowerblue'
    if label == "Lookahead 2":
        return 'darkblue'
    if label == "No Subgoals":
        return 'grey'
    if label == "Full Decomp":
        return 'hotpink'
    if label == "Full Decomp 3":
        return 'hotpink'
    if label == "Full Decomp 4":
        return "purple"
    # if 'Myopic' in label:
    #     return [43/255,108/255,162/255,]
    # elif 'Lookahead' in label:
    #     return [150/255,43/255,162/255,]
    # elif 'Best First' in label:
    #     return [42/255,132/255,94/255,]
    # else:
    #     return [174/255,55/255,4/255,]

In [ ]:
def plot_data(fdf, x_axis, y_axis, x_label, y_label, title, x_scale='linear', y_scale='log', legend=True):
    plt.figure(figsize=(7, 7))

    for label in tqdm(fdf['label first'].unique()):
        ag_df = fdf[fdf['label first'] == label]
        ag_df = ag_df.sort_values(by=x_axis)
        jitter = np.random.normal(0, 0.5, ag_df.shape[0])

        plt.scatter(ag_df[x_axis]+jitter, ag_df[y_axis], alpha=0.5, marker=get_shape(label), color=get_colors(label))

        means = []
        cis = []

        for size in ag_df[x_axis].unique():
            mean, ci, true_mean = bootstrap_over_runs(ag_df[ag_df[x_axis] == size], y_axis, stat_function=np.mean, show_tqdm=False)
            means.append(mean)
            cis.append(ci)

        plt.plot(ag_df[x_axis].unique(), means, color=get_colors(label), marker=get_shape(label), markersize=10, label=label)
        plt.fill_between(ag_df[x_axis].unique(), [c[0] for c in cis], [c[1] for c in cis], alpha=0.2, color=get_colors(label))

    plt.yscale(y_scale)
    plt.xscale(x_scale)
    if legend: 
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    # plt.show()
    # return the plot object
    return plt.gcf()

## Analyses

### Solving these towers by doing action level search alone is hard.

In [ ]:
df[df['label'] == 'No Subgoals'].sample(3)

In [ ]:
print("Mean success rate for no subgoals")
mean, ci, true_mean = bootstrap_over_runs(fdf[fdf['label first'] == 'No Subgoals'], 'perfect last', stat_function = np.mean)
print("True Mean: {}, Bootstraped Mean: {}, CI: {}".format(true_mean, mean, ci))

In [ ]:
print("Mean Action Level Cost for no subgoals")
mean, ci, true_mean = bootstrap_over_runs(df[df['label'] == 'No Subgoals'], 'cost', stat_function = np.mean)
print("True Mean: {}, Bootstrapped Mean: {}, CI: {}".format(true_mean, mean, ci))

#### Relation to tower size

In [ ]:
# smallest and largest worlds cost and success rate
# bootstrap
print("Smallest worlds")
print("Smallest size: {}".format(fdf['world_size item'].min()))
mean, ci, true_mean = bootstrap_over_runs(fdf[fdf['world_size item'] == fdf['world_size item'].min()], 'cost sum', stat_function = np.mean)
print("*Cost sum*: True Mean: {}, Bootstrapped Mean: {}, CI: {}".format(true_mean, mean, ci))
mean, ci, true_mean = bootstrap_over_runs(fdf[fdf['world_size item'] == fdf['world_size item'].min()], 'perfect last', stat_function = np.mean)
print("*Perfect*: True Mean: {}, Bootstrapped Mean: {}, CI: {}".format(true_mean, mean, ci))
print("Largest world")
print("Largest size: {}".format(fdf['world_size item'].max()))
mean, ci, true_mean = bootstrap_over_runs(fdf[fdf['world_size item'] == fdf['world_size item'].max()], 'cost sum', stat_function = np.mean)
print("*Cost sum*: True Mean: {}, Bootstrapped Mean: {}, CI: {}".format(true_mean, mean, ci))
mean, ci, true_mean = bootstrap_over_runs(fdf[fdf['world_size item'] == fdf['world_size item'].max()], 'perfect last', stat_function = np.mean)
print("*Perfect*: True Mean: {}, Bootstrapped Mean: {}, CI: {}".format(true_mean, mean, ci))



### The use of visual subgoals can greatly reduce the action planning costs over not using subgoals, with full decomposition reducing it most.

In [ ]:
agent_summary = pd.DataFrame(columns = ['mean action level cost', 'mean subgoal level cost', 'mean success rate', 'CI action level cost', 'CI subgoal level cost', 'CI success rate'], index = fdf['label first'].unique())
for label in tqdm(fdf['label first'].unique()):
    label_dict = {}
    # get action level cost
    mean, ci, true_mean = bootstrap_over_runs(fdf[fdf['label first'] == label], 'cost sum', stat_function = np.mean, show_tqdm = False)
    label_dict['mean action level cost'] = true_mean
    label_dict['CI action level cost'] = ci
    # get subgoal level cost
    mean, ci, true_mean = bootstrap_over_runs(fdf[fdf['label first'] == label], 'total_cost sum', stat_function = np.mean, show_tqdm = False)
    label_dict['mean subgoal level cost'] = true_mean
    label_dict['CI subgoal level cost'] = ci
    # get success rate
    mean, ci, true_mean = bootstrap_over_runs(fdf[fdf['label first'] == label], 'perfect last', stat_function = np.mean, show_tqdm = False)
    label_dict['mean success rate'] = true_mean
    label_dict['CI success rate'] = ci
    # add to dataframe
    agent_summary.loc[label] = label_dict
display(agent_summary)

### But it comes with a cost of choosing subgoals—which makes lookahead look like the most promising strategy

Scatter plot of action, subgoal levels costs across different agents.

In [ ]:
fdf.sample(3)

### This is mediated by the size of the problem—for smaller problem full decomp might better.

In [ ]:
plot = plot_data(fdf, 'world_size item', 'cost sum', 'World Size', 'Action Level Cost', 'Action Level Cost vs World Size')

In [ ]:
plot = plot_data(fdf, 'world_size item', 'total_cost sum', 'World Size', 'Subgoal Level Cost\n(states explored)', 'Subgoal Level Cost (states) vs World Size')

In [ ]:
plot = plot_data(fdf, 'world_size item', 'all_sequences_count sum', 'World Size', 'Subgoal Level Cost\n(decompositions explored)', 'Subgoal Level Cost (decompositions) vs World Size')

In [ ]:
plot = plot_data(fdf, 'world_size item', 'perfect last', 'World Size', 'Success Rate', 'Success Rate vs World Size', y_scale='linear')

### High cost avoidance ($\lambda$) indeed leads to cheaper solutions, but also getting stuck more often.

In [ ]:
plot = plot_data(fdf, 'c_weight first', 'cost sum', '$\lambda$', 'Action Level Cost', 'Action Level Cost vs $\lambda$', x_scale='log')

In [ ]:
plot = plot_data(fdf, 'c_weight first', 'total_cost sum', '$\lambda$', 'Subgoal Level Cost\n(states evaluated)', 'Subgoal Level Cost (states) vs $\lambda$', x_scale='log')

In [ ]:
plot = plot_data(fdf, 'c_weight first', 'all_sequences_count sum', '$\lambda$', 'Subgoal Level Cost\n(decompositions evaluated)', 'Subgoal Level Cost (decompositions) vs $\lambda$', x_scale='log')

In [ ]:
plot = plot_data(fdf, 'c_weight first', 'perfect last', '$\lambda$', 'Success Rate', 'Success Rate vs $\lambda$', x_scale='log', y_scale='linear')

### Paradoxically, avoiding action level costs leads to more subgoal selection and therefore higher costs.

# Debugging

In [ ]:
# get world where no subgoal fails
fdf[(fdf['label first'] == 'No Subgoals') & (fdf['perfect last'] != 1)]['world item'].value_counts()

In [ ]:
fdf[fdf['label first'] == 'No Subgoals']

In [ ]:
fdf[fdf['world item'] == 'Blockworld_111']

In [ ]:
seq = df[(df['world'] == 'Blockworld_111') & (df['label'] == 'No Subgoals')]['_chosen_subgoal_sequence'].dropna().tail(1).item()

In [ ]:
seq[0].subgoals

In [ ]:
id = df[(df['world'] == 'Blockworld_111') & (df['label'] == 'No Subgoals') & (df['c_weight'] == 0.0)]['run_ID'].head(1).item()

In [ ]:
df[df['run_ID'] == id]

In [ ]:
df[df['run_ID'] == id].tail(1)._world.item().current_state.blockmap

In [ ]:
seq.visualize()